In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('data/resultats.csv')

In [3]:
df['departement'] = df['departement'].str.pad(2, fillchar='0')
df['circo'] = df['circo'].map(str).str.pad(3,fillchar='0')
df['code'] = df['departement'] + df['circo']

df = df.set_index('code')

In [4]:
for c in df:
    if any(c.startswith(pref) for pref in ['Prénom', 'N°Panneau', 'Sexe']):
        del df[c]

In [5]:
noms = pd.concat([df[c] for c in df if c.startswith('Nom')], ignore_index=True).unique()

In [6]:
totaux = df[['departement_libelle', 'circo_libelle', 'inscrits', 'abstentions', 'votants', 'blancs', 'nuls', 'exprimes']].sort_index()

In [7]:
reste = df.iloc[:, 2+len(totaux.columns):]

In [8]:
res = []
for i in range(len(reste.columns)//2):
    colonnes = reste.iloc[:, 2*i:2*i+2]
    colonnes.columns = ['nom', 'votes']
    col = colonnes.reset_index().set_index(['code', 'nom'])['votes']
    res.append(col)

In [9]:
votes = pd.concat(res).unstack().sort_index()

In [10]:
exprimes = votes.divide(totaux['exprimes'], axis=0)

In [44]:
rang = votes.gt(votes['MÉLENCHON'], axis=0).sum(axis=1) + 1

In [46]:
qualifie = (votes['MÉLENCHON'] / totaux['inscrits']) >= 0.125

In [47]:
topology = json.load(open('data/circos_topo.json'))

In [48]:
for geometry in topology['objects']['circos']['geometries']:
    circo = geometry['properties']['ID']
    geometry['properties']['votes'] = exprimes.loc[circo].to_dict()
    geometry['properties']['votes']['rang'] = int(rang.loc[circo])
    geometry['properties']['votes']['qualifie'] = bool(qualifie.loc[circo])

In [49]:
json.dump(topology, open('data/circos_exprimes.json', 'w'))